# Diagnóstico de la Regresión Lineal

### Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

### Creación de dataset

In [ ]:
def load_boston():
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = raw_df.values[1::2, 2]
    return {'data': data, 'target': target, 'DESCR': 'boston dataset', 'feature_names': ['CRIM','ZN','INDUS','CHAS', 'NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']}

### Cargar dataset "Boston" con SKLEARN

In [ ]:
data = load_boston()

In [ ]:
# Mostrar las claves del diccionario
data.keys()

In [ ]:
# Mostrar la descripción de los datos.
print(data['DESCR'])

In [ ]:
# Las variables explicativas.
X = data['data']
header = data['feature_names']

In [ ]:
# La variable respuesta.
Y = data['target']
Y = Y.reshape(-1, 1)

#### Convierte los datos en un DataFrame y luego explora:

In [ ]:
df = pd.DataFrame(np.append(X,Y,axis = 1))
df.columns = list(header)+['PRICE']

In [ ]:
df.head(5)

In [ ]:
# Estadística descriptiva de las columnas.
df.describe()

In [ ]:
# Matriz de correlación por pares.
np.round(df.corr(),2)

# Matriz de Correlación por Pares

La **matriz de correlación por pares** es una matriz que muestra los coeficientes de correlación entre todas las posibles combinaciones de un conjunto de variables. Cada celda en la matriz representa la correlación entre dos variables diferentes. La correlación mide la relación lineal entre dos variables, con valores que van desde -1 hasta 1:

- **1** indica una correlación positiva perfecta, donde las variables aumentan o disminuyen juntas.
- **0** indica que no hay correlación lineal entre las variables.
- **-1** indica una correlación negativa perfecta, donde una variable aumenta mientras la otra disminuye.

## Contexto y Uso

En análisis de datos, la matriz de correlación es útil para identificar relaciones entre variables. Es comúnmente utilizada en estadística, ciencia de datos y aprendizaje automático para:

- Detectar multicolinealidad.
- Identificar relaciones lineales entre variables.
- Seleccionar características en modelos predictivos.

## Ejemplo en Python con Pandas

Aquí hay un ejemplo de cómo calcular una matriz de correlación por pares usando `pandas` en Python:

```python
import pandas as pd

# Crear un DataFrame de ejemplo
data = {
    'Variable1': [1, 2, 3, 4, 5],
    'Variable2': [10, 20, 30, 40, 50],
    'Variable3': [5, 4, 3, 2, 1]
}

df = pd.DataFrame(data)

# Calcular la matriz de correlación
correlation_matrix = df.corr()

print(correlation_matrix)


## Salida Esperada

La salida es una matriz simétrica donde cada elemento representa la correlación entre un par de variables:



            Variable1  Variable2  Variable3
Variable1        1.0        1.0       -1.0
Variable2        1.0        1.0       -1.0
Variable3       -1.0       -1.0        1.0



En este ejemplo:

- `Variable1` y `Variable2` tienen una correlación perfecta positiva (1.0).
- `Variable1` y `Variable3`, así como `Variable2` y `Variable3`, tienen una correlación perfecta negativa (-1.0).

## Tipos de Correlación

Pandas utiliza por defecto la **correlación de Pearson**, que mide la relación lineal. También se pueden calcular otros tipos de correlación como:

- **Spearman**: Para relaciones monótonas.
- **Kendall**: Para evaluar la concordancia entre variables.

Puedes especificar el método al calcular la matriz de correlación:

```python
correlation_matrix = df.corr(method='spearman')


In [ ]:
# Visualice la matriz de correlación.
sns.heatmap(df.corr(),cmap='coolwarm')
plt.show()

In [ ]:
# Visualizar RM vs PRICE.
plt.scatter(X[:,5],Y[:,0],c = 'g',s=15,alpha=0.5)
plt.xlabel('RM')
plt.ylabel('PRICE')
plt.show()

### Entrenar regresión lineal

In [ ]:
# Train.
lm = LinearRegression(fit_intercept=True)
lm.fit(X,Y)

In [ ]:
# Intercepto
lm.intercept_

In [ ]:
# El resto de coeficientes (parámetros).
lm.coef_

In [ ]:
# Mostrar los parámetros como un DataFrame.
parametersDF = pd.DataFrame(lm.coef_,index=['Parameter Value'],columns=header)
parametersDF['Intercept'] = lm.intercept_[0]
parametersDF

### Diagnóstico

In [ ]:
# Predicción dentro de la muestra.
predY = lm.predict(X)

In [ ]:
# Mostrar Y real vs Y previsto.
plt.scatter(Y,predY,c = 'blue', s=15, alpha=0.5)
plt.xlabel('REAL PRICE')
plt.ylabel('PREDICTED PRICE')
plt.show()

In [ ]:
# Calcular la correlación entre la Y real y la Y prevista.
pd.Series(Y[:,0]).corr(pd.Series(predY[:,0]))

In [ ]:
# Coeficiente de determinación (R^2):
lm.score(X,Y)

### Pruebas dentro y fuera de la muestra:

In [ ]:
# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
# predY_in = predicción de Y dentro de la muestra.
# predY_out = predicción de Y fuera de la muestra.
lm = LinearRegression()
lm.fit(X_train,Y_train)
Y_pred_train = lm.predict(X_train)
Y_pred_test = lm.predict(X_test)

In [ ]:
print('MSE dentro de la muestra es      : ' + str(metrics.mean_squared_error(Y_train, Y_pred_train)))
print('MSE fuera de la muestra es       : ' + str(metrics.mean_squared_error(Y_test, Y_pred_test)))
print('-'*50)
print('RMSE dentro de la muestra es     : ' + str(np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))))
print('RMSE fuera de la muestra es      : ' + str(np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))))


### Explicación:

Este fragmento de código evalúa el rendimiento de un modelo de aprendizaje automático comparando los valores predichos con los valores reales, tanto para el conjunto de datos de entrenamiento (dentro de la muestra) como para el conjunto de datos de prueba (fuera de la muestra).

1. **Error Cuadrático Medio (MSE)**:
   - **Dentro de la muestra (In-sample MSE)**: Se calcula utilizando los datos de entrenamiento (`Y_train`) y las predicciones correspondientes (`Y_pred_train`). Esto mide el error promedio de las predicciones dentro del conjunto de entrenamiento.
   - **Fuera de la muestra (Out-of-sample MSE)**: Se calcula utilizando los datos de prueba (`Y_test`) y las predicciones correspondientes (`Y_pred_test`). Esto mide el error promedio de las predicciones en el conjunto de prueba, evaluando la capacidad del modelo para generalizar a datos nuevos.

2. **Raíz del Error Cuadrático Medio (RMSE)**:
   - **Dentro de la muestra (In-sample RMSE)**: Es la raíz cuadrada del MSE dentro de la muestra. Proporciona una medida del error en las mismas unidades que la variable objetivo, lo que puede ser más interpretable que el MSE.
   - **Fuera de la muestra (Out-of-sample RMSE)**: Es la raíz cuadrada del MSE fuera de la muestra, proporcionando una medida similar para el conjunto de prueba.

### Contexto
Cuando entrenas un modelo de aprendizaje automático, se suelen utilizar dos conjuntos de datos:
- **Conjunto de entrenamiento**: Utilizado para ajustar el modelo.
- **Conjunto de prueba**: Utilizado para evaluar qué tan bien el modelo generaliza a datos nuevos.

### Valores Proporcionados
1. **In-sample MSE (Error Cuadrático Medio dentro de la muestra)**: 
   - **Valor**: 20.184336639873152
   - **Interpretación**: Este valor indica el error promedio que comete el modelo al predecir los datos dentro del conjunto de entrenamiento. Un valor más bajo significa que el modelo ajusta bien los datos de entrenamiento.

2. **Out-of-sample MSE (Error Cuadrático Medio fuera de la muestra)**: 
   - **Valor**: 28.40585481050845
   - **Interpretación**: Este valor muestra el error promedio del modelo al predecir los datos en el conjunto de prueba. Este valor es generalmente más alto que el MSE dentro de la muestra porque el modelo está prediciendo sobre datos que no ha visto antes. Si este valor es significativamente mayor que el MSE dentro de la muestra, puede indicar que el modelo podría estar sobreajustado (overfitted).

3. **In-sample RMSE (Raíz del Error Cuadrático Medio dentro de la muestra)**: 
   - **Valor**: 4.492698146979513
   - **Interpretación**: Es la raíz cuadrada del MSE dentro de la muestra. Proporciona el error en las mismas unidades que la variable objetivo, lo que puede ser más fácil de interpretar. Un valor más bajo significa que el modelo es más preciso con los datos de entrenamiento.

4. **Out-of-sample RMSE (Raíz del Error Cuadrático Medio fuera de la muestra)**: 
   - **Valor**: 5.3297143272888885
   - **Interpretación**: Es la raíz cuadrada del MSE fuera de la muestra. Este valor también está en las mismas unidades que la variable objetivo y, al ser mayor que el RMSE dentro de la muestra, sugiere que el modelo tiene más error al predecir sobre datos nuevos.

### Resumen General
- **MSE** y **RMSE** más bajos indican un mejor rendimiento del modelo.
- En este caso, los valores de error fuera de la muestra (tanto MSE como RMSE) son mayores que los valores dentro de la muestra, lo que es esperado. Sin embargo, la diferencia no es drástica, lo que sugiere que el modelo generaliza razonablemente bien.
- Si la diferencia fuera mayor, podríamos sospechar que el modelo está sobreajustado a los datos de entrenamiento.


### Análisis de Residuales en Regresión Lineal

El análisis de residuales es una parte fundamental de la evaluación de un modelo de regresión lineal (RL). Los **residuales** son las diferencias entre los valores observados y los valores predichos por el modelo. Al analizar los residuales, puedes obtener información sobre el ajuste del modelo y detectar posibles problemas que podrían afectar su interpretación y validez.

#### ¿Qué son los residuales?
Los residuales (o errores) se calculan como:

\[ \text{Residual} = Y_{\text{observado}} - Y_{\text{predicho}} \]

Donde:
- \( Y_{\text{observado}} \) es el valor real o observado de la variable dependiente.
- \( Y_{\text{predicho}} \) es el valor predicho por el modelo de regresión.

#### Objetivos del Análisis de Residuales
El análisis de residuales tiene varios objetivos:
1. **Verificar la linealidad**: Los residuales deben estar distribuidos de manera aleatoria alrededor de cero, lo que indica que la relación entre las variables independientes y dependientes es lineal.
2. **Evaluar la homocedasticidad**: Esto significa que la varianza de los residuales debe ser constante a lo largo de todos los niveles de la variable independiente. Si no es así (es decir, si hay heterocedasticidad), puede haber problemas con el modelo.
3. **Detectar normalidad**: Los residuales deben seguir una distribución normal. Si no lo hacen, podría afectar la validez de las pruebas estadísticas.
4. **Identificar puntos atípicos**: Los residuales muy grandes pueden indicar la presencia de outliers o puntos atípicos que podrían influir de manera desproporcionada en el modelo.
5. **Detección de autocorrelación**: En el contexto de series temporales o datos ordenados, se verifica si los residuales están correlacionados entre sí, lo que podría indicar que el modelo no ha capturado alguna estructura en los datos.

#### Métodos Comunes de Análisis de Residuales
1. **Gráfico de Residuales vs. Valores Ajustados**: Este gráfico muestra los residuales en el eje y y los valores predichos en el eje x. Deberías observar una dispersión aleatoria sin patrones claros.
   - **Patrón en forma de U o arco**: Indica que la relación entre las variables podría no ser lineal.
   - **Patrón en embudo**: Sugiere heterocedasticidad.

2. **Gráfico Q-Q (Quantile-Quantile)**: Este gráfico compara la distribución de los residuales con una distribución normal teórica. Si los residuales son normales, los puntos deberían alinearse en una línea recta.
   - **Desviaciones significativas de la línea**: Indican que los residuales no son normales.

3. **Histograma de los Residuales**: Un histograma te permite visualizar la distribución de los residuales. Debería aproximarse a una campana simétrica (distribución normal).
   - **Sesgo en un lado**: Puede indicar problemas de normalidad.

4. **Durbin-Watson**: Una prueba estadística utilizada para detectar la autocorrelación en los residuales de un modelo de regresión.

#### ¿Qué hacer si encuentras problemas?
- **Linealidad**: Si los residuales muestran un patrón no lineal, podrías considerar transformar las variables o usar un modelo no lineal.
- **Heterocedasticidad**: Si detectas heterocedasticidad, podrías probar transformar la variable dependiente, usar métodos robustos a la heterocedasticidad, o emplear un modelo de regresión ponderada.
- **Normalidad**: Si los residuales no son normales, podrías considerar transformaciones de las variables o utilizar métodos que no asuman normalidad, como la regresión cuantílica.
- **Outliers**: Identificar y analizar los outliers para decidir si se deben eliminar, o si debes ajustar el modelo para considerar su impacto.
- **Autocorrelación**: Si los residuales están autocorrelacionados, podrías necesitar un modelo más complejo, como una regresión con términos de autoregresión o modelos de series temporales.

In [ ]:
# Calcular residual.
residual = Y_train - Y_pred_train

In [ ]:
# Q: ¿Puedes comprobar "visualmente" que la media = 0 y la varianza = constante?
plt.scatter(Y_train,residual,c = 'red', s=15, alpha=0.5)
plt.xlabel('Y')
plt.ylabel('Residual')
plt.title('Residual')
plt.show()

In [ ]:
# Q: ¿Los residuos se distribuyen normalmente centrados alrededor de 0?
sns.histplot(residual, bins=50, color='green').set_title("Residual Histogram")
plt.show()

#### Dado un nuevo conjunto de valores para las variables explicativas, prediga la respuesta:
- CRIM : 0,03
- ZN : 0,0
- INDUS : 13,0
- CHAS : 0,0
- NOX : 0,4
- RM : 4,3
- AGE : 23,5
- DIS : 1,9
- RAD : 1,0
- TAX : 273,0
- PTRATIO : 18,0
- B : 380,0
- LSTAT : 7,5

In [ ]:
X_new = np.array([0.03, 0.0, 13.0, 0.0, 0.4, 4.3, 23.5, 1.9, 1.0, 273.0, 18.0, 380.0, 7.5]).reshape(1,-1)  # Reshaped as a row.
Y_pred_new = lm.predict(X_new)
print(np.round(Y_pred_new[0,0],3))

## Clasificación con regresión logística

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
# Cargar datos
data = load_breast_cancer()

In [ ]:
# Mostrar la descripción.
print(data.DESCR)

In [ ]:
# Explanatory variables.
X = data['data']
print(data['feature_names'])

In [ ]:
X.shape

In [ ]:
# Variable de respuesta.
# Vuelva a etiquetar de modo que 0 = 'benigno' y 1 = maligno.
Y = 1 - data['target']
label = list(data['target_names'])
label.reverse()
print(label)

In [ ]:
# Visualizar la frecuencia.
ser = pd.Series(Y)
table = ser.value_counts()
table = table.sort_index()         # Debe ordenarse para un etiquetado correcto.
raw_data = {'x': label, 'y': table.values}
sns.barplot(x='x', y='y', data=raw_data)
plt.show()

### Test y Train split

In [ ]:
# Divida el conjunto de datos en entrenamiento y prueba.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=1234)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
# Entrenar y predecir.
LL = LogisticRegression(solver='liblinear',max_iter=200)
LL.fit(X_train,Y_train)
Y_pred_test = LL.predict(X_test) # Predicción fuera de la muestra.

In [ ]:
# Confusion matrix.
conf_mat = metrics.confusion_matrix(Y_test,Y_pred_test)
print(conf_mat)

In [ ]:
# Accuracy, Sensitivity, Specificity y Precision usando la confusion matrix.
accuracy = (conf_mat[0,0] + conf_mat[1,1])/np.sum(conf_mat)
sensitivity = conf_mat[1,1]/(conf_mat[1,0]+conf_mat[1,1])
specificity = conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
precision = conf_mat[1,1]/(conf_mat[0,1]+conf_mat[1,1])
print('Accuracy    = {}'.format(np.round(accuracy,3)))
print('Sensitvity  = {}'.format(np.round(sensitivity,3)))
print('Specificity = {}'.format(np.round(specificity,3)))
print('Precision   = {}'.format(np.round(precision,3)))

In [ ]:
# Alternativa.
accuracy = metrics.accuracy_score(Y_test,Y_pred_test)                      # Alternative way to calculate the accuracy.
sensitivity = metrics.recall_score(Y_test,Y_pred_test)
precision = metrics.precision_score(Y_test,Y_pred_test)
print('Accuracy    = {}'.format(np.round(accuracy,3)))
print('Sensitvity  = {}'.format(np.round(sensitivity,3)))
print('Precision   = {}'.format(np.round(precision,3)))

#### Límite de corte (umbral):

In [ ]:
# Ahora, prediga la probabilidad de Y = 1.
Y_pred_test_prob=LL.predict_proba(X_test)[:,1]

In [ ]:
# Se puede cambiar el valor de corte a voluntad
cutoff = 0.7 # el valor de corte puede ser un valor entre 0 y 1.
Y_pred_test_val = (Y_pred_test_prob > cutoff).astype(int)
conf_mat = metrics.confusion_matrix(Y_test,Y_pred_test_val)
print(conf_mat)

In [ ]:
accuracy = (conf_mat[0,0] + conf_mat[1,1])/np.sum(conf_mat)
sensitivity = conf_mat[1,1]/(conf_mat[1,0]+conf_mat[1,1])
specificity = conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
precision = conf_mat[1,1]/(conf_mat[0,1]+conf_mat[1,1])
print('Accuracy    = {}'.format(np.round(accuracy,3)))
print('Sensitvity  = {}'.format(np.round(sensitivity,3)))
print('Specificity = {}'.format(np.round(specificity,3)))
print('Precision   = {}'.format(np.round(precision,3)))

### Curva ROC

In [ ]:
# Initialize.
cutoff_grid = np.linspace(0.0,1.0,100)
TPR = []                                                   # True Positive Rate.
FPR = []                                                   # False Positive Rate.

In [ ]:
# Populate the TP and FP lists.
for cutoff in cutoff_grid:
    Y_pred_test_val = (Y_pred_test_prob > cutoff).astype(int)
    conf_mat = metrics.confusion_matrix(Y_test,Y_pred_test_val)
    sensitivity = conf_mat[1,1]/(conf_mat[1,0]+conf_mat[1,1])
    specificity = conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
    TPR.append(sensitivity)
    FPR.append(1-specificity)  

In [ ]:
# Visualizar.
plt.plot(FPR,TPR,c='red',linewidth=1.0)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
# Calculate the TPR and FPR using a Scikit Learn function.
FPR, TPR, cutoffs = metrics.roc_curve(Y_test,Y_pred_test_prob,pos_label=1)      # positive label = 1.

In [ ]:
# Visualize.
plt.plot(FPR,TPR,c='red',linewidth=1.0)
plt.xlabel('False Positive')
plt.ylabel('True Positive')
plt.title('ROC Curve')
plt.show()

In [ ]:
# AUC.
auc = metrics.roc_auc_score(Y_test,Y_pred_test_prob)
print('AUC  = {}'.format(np.round(auc,3)))